In [28]:
!pip install -U sentence-transformers
!pip install transformers



In [3]:
from google.colab import files
uploaded = files.upload()


Saving Diverse_Policy_Statements.csv to Diverse_Policy_Statements (1).csv


In [4]:
import pandas as pd

df = pd.read_csv("Diverse_Policy_Statements.csv")
df.head()


,Diverse Policy Statements
0,Public-private partnerships were formed in bio...
1,The pension age was lowered to address unemplo...
2,Tariffs on imported steel were lifted to reduc...
3,Funding for 5G infrastructure development was ...
4,Job training programs were expanded for displa...


In [5]:
from sentence_transformers import SentenceTransformer

# 1. Modeli yükle
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Politika cümlelerini listeye çevir
sentences = df['Diverse Policy Statements'].tolist()

# 3. Cümlelerden embedding çıkar (vektörleştir)
embeddings = model.encode(sentences, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
!pip install faiss-cpu



In [25]:
import faiss
import numpy as np

# 1. Embedding'leri normalize et (cosine similarity için)
embeddings = np.array(embeddings).astype('float32')
faiss.normalize_L2(embeddings)

# 2. FAISS index oluştur (cosine similarity için flat index)
index = faiss.IndexFlatIP(embeddings.shape[1])  # IP = Inner Product (cosine similarity için)
index.add(embeddings)

# 3. Örnek bir soru
query = "What are the government’s responses to unemployment?"
query_embedding = model.encode([query])
faiss.normalize_L2(query_embedding)

# 4. En benzer 15 cümleyi getir
top_k = 15.  #if top_k = 200 (full data), problems: Token overflow, low quality, loss of focus
D, I = index.search(query_embedding, top_k)

retrieved = []
seen = set()
for i in I[0]:
    if sentences[i] not in seen:
        retrieved.append(sentences[i])
        seen.add(sentences[i])


# 5. Sonuçları yazdır
for i in I[0]:
    print(f"- {sentences[i]}")


- Unemployment benefits were extended during the crisis.
- Unemployment benefits were extended during the crisis.
- Unemployment benefits were extended during the crisis.
- Unemployment benefits were extended during the crisis.
- Unemployment benefits were extended during the crisis.
- Unemployment benefits were extended during the crisis.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- The pension age was lowered to address unemployment rates.
- Job training programs were expanded for displaced workers.


In [21]:
#Count the number of repetition for the given sentence, if any.
from collections import Counter
Counter(sentences)["Unemployment benefits were extended during the crisis."]


6

In [26]:
# Filter the repeated sentences
retrieved = []
seen = set()
for i in I[0]:
    if sentences[i] not in seen:
        retrieved.append(sentences[i])
        seen.add(sentences[i])


In [27]:
#Check whether filter works
for sentence in retrieved:
    print("-", sentence)


- Unemployment benefits were extended during the crisis.
- The pension age was lowered to address unemployment rates.
- Job training programs were expanded for displaced workers.


In [29]:
# Generation Part: retrieved context + kullanıcı sorusu → LLM → akıllı cevap

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
rag_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [38]:
# Soru ve context
question = "What are the government’s responses to unemployment?"
context = "\n".join(retrieved[:10])  # En fazla 10 farklı cümle

# Prompt dizaynı
prompt = f"""Answer the following question based on the given context.

Context:
{context}

Question: {question}
Answer:"""

# Tokenize et
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

# Cevabı üret
outputs = rag_model.generate(**inputs, max_new_tokens=100)

# Cevabı çöz
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)

# Yanıt fazla kısa ve genel. flan-t5-base modeli sınırlı kapasiteye sahip olduğu için derinlemesine yanıt üretme konusunda kısıtlı kalabiliyor.
# Prompt'ı geliştir.


benefits were extended


In [39]:
# Soru ve context
question = "What are the government’s responses to unemployment?"
context = "\n".join(retrieved[:10])  # En fazla 10 farklı cümle

# Prompt dizaynı
prompt = f"""You are a policy assistant. Based on the following policy statements, summarize how governments have responded to unemployment.

Policy Statements:
{context}

Question: {question}
Answer in 1–2 sentences:"""


# Tokenize et
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

# Cevabı üret
outputs = rag_model.generate(**inputs, max_new_tokens=100)

# Cevabı çöz
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)


benefits were extended during the crisis. The pension age was lowered to address unemployment rates. Job training programs were expanded for displaced workers.
